## AutoGrad : 

It's can automatically differentiation method.

for example if we have : 

let $a, b, x, y \in \mathbb{R} $

$$ y = ax + b \Longrightarrow \frac{\partial y}{\partial a} = x $$

and

$$ y = ax + b \Longrightarrow \frac{\partial y}{\partial b} = 1 $$


$$ y = a + b \Longrightarrow \frac{\partial y}{\partial a} = 1 $$

$$ y = a + b \Longrightarrow \frac{\partial y}{\partial b} = 1 $$


$$ y = a * b \Longrightarrow \frac{\partial y}{\partial a} = b $$

$$ y = a * b \Longrightarrow \frac{\partial y}{\partial b} = a $$

**Vectors :**

$$ y = x^Tw \Longrightarrow \frac{\partial y}{\partial w} = x^T $$

$$ y = x^Tw \Longrightarrow \frac{\partial y}{\partial x} = w^T $$

**Activation**

Let 

$$ y = \sigma (z)$$

where 

$$\sigma(z) = \frac{1}{1 + \exp^{-z}}$$ 

for $z \in \mathbb{R} \hspace{0.5cm} \sigma^{'}(z) = \sigma(z)(1- \sigma(z))$

if $z = x^Tw$ thus, 

$$ \frac{\partial \sigma(z)}{\partial w} =  \frac{\partial \sigma(z)}{\partial z}\frac{\partial z}{\partial w} \mid z = x^Tw$$ 

$$ =   \sigma(z)(1- \sigma(z))w^T$$

$$ =   \sigma(x^Tw)(1- \sigma(x^Tw))w^T$$


### For Real Value : 


$$ y = a + b \Longrightarrow \begin{equation}
\left\{\begin{split}
\frac{\partial y}{\partial a} = 1 \\
\frac{\partial y}{\partial b} = 1 \\
\end{split}\right.
\end{equation}$$


$$ y = f_1 \circ f_2 \circ ... \circ f_n(x) \Longrightarrow \frac{\partial y}{\partial x} = \frac{\partial y(f_1)}{\partial f_1} \frac{\partial f_1(f_2)}{\partial f_2} ... \frac{\partial f_n(x)}{\partial x} $$

In [302]:
class Value:

    def __init__(self, data: float, _childs : tuple[Value] = () , ops: str =  'None'):
        self.data = data
        self.grad = 0
        self.ops = ops
        self._prev = set(_childs)
        self._backword = lambda : None  

    def __add__(self, other: Value) -> Value:
        out =  Value(self.data + other.data, (self, other), ops='+')    

        def _backword():
            self.grad = out.grad
            other.grad = out.grad

        out._backword = _backword
        
        return out
    
    def __mul__(self, other: Value) -> Value:
        out = Value(self.data * other.data, (self, other), ops='*')

        def _backword():
            self.grad = other.data
            other.grad = self.data
            
        out._backword = _backword
        
        return out     

    def relu(self):
        out = Value(0 if self.data <= 0 else self.data, (self,), 'ReLU')

        out.grad = 1

        print(out)
        
        def _backword():
            self.grad = (out.data > 0)*out.grad
            
        out._backword = _backword

        return out

    def backward(self):

        self.nodes = []
        visited = set()
        def get_nodes(value: Value):
            if value not in visited:
                visited.add(value)
                
                self.nodes.append(value)
                
                for child in value._prev:
                    get_nodes(child)
                    
                
        get_nodes(self)
        
        self.grad = 1

        for v in self.nodes:
            v._backword()

        
    def __repr__(self) -> str:
        return f"Value(data={self.data}, grad={self.grad}, ops={self.ops})"

In [303]:
a = Value(3)

In [304]:
b = Value(2)

In [305]:
y = a.relu() * b

Value(data=3, grad=1, ops=ReLU)


In [306]:
y.backward()

In [307]:
a.grad

2

In [308]:
y.nodes

[Value(data=6, grad=1, ops=*),
 Value(data=2, grad=3, ops=None),
 Value(data=3, grad=2, ops=ReLU),
 Value(data=3, grad=2, ops=None)]

In [ ]:
class Tensor: